In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
def query_enhance(input_query="George Enescu"):
    chat = [
        {"role": "system",
         "content": "Ești un expert în procesarea interogărilor. Reformulează interogarea pentru a include subiecte relevante și posibile sinonime. Fii cât mai detaliat."},
        {"role": "user", "content": f"Interogare inițială: \"{input_query}\"\nReformulează interogarea:"},
    ]
    
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, system_message="")
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")
    
    # Create attention mask
    attention_mask = (inputs != tokenizer.pad_token_id).long()
    
    # Generate output
    outputs = model.generate(
        input_ids=inputs,
        attention_mask=attention_mask,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id  # Ensure padding is treated as the end of a sequence
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Post-process the output to extract only the enhanced query
    if "assistant" in result:
        enhanced_query = result.split("assistant")[1].strip()
    else:
        enhanced_query = result.strip()
    
    
    return enhanced_query

query_enhance()

'Reformularea interogării: „Căutați orice informații relevante despre George Enescu, un compozitor, violonist și pedagog românesc, cunoscut pentru contribuțiile sale la muzică și educație.”'

# Initialize IR endpoints

In [18]:
from py4j.java_gateway import JavaGateway

# Connect to the Java Gateway
gateway = JavaGateway()
ir_system = gateway.entry_point

#### CREATE INDEX #####
# folderPath = "src/main/resources"
# ir_system.index(folderPath)
#### CREATE INDEX #####

# Tested queries and expected docs

In [43]:
# Test 1
query1 = "Ce pot vizita în România?"

# Expected documents:
docs1 = ["doc11.txt", "doc13.txt", "doc19.txt"]
# Test 2
query2 = "Care sunt mâncărurile tradiționale românești?"

# Expected documents:
docs2 = ["doc17.txt", "doc27.txt", "doc11.txt"]
# Test 3
query3 = "Cine sunt artiștii români celebri?"

# Expected documents:
docs3 = ["doc22.txt", "doc25.txt", "doc27.txt"]
# Test 4
query4 = "Ce știi despre Castelul Bran și Dracula?"

# Expected documents:
docs4 = ["doc23.txt", "doc21.txt", "doc19.txt"]
# Test 5
query5 = "Care este limbajul de programare utilizat în învățarea automată?"

# Expected documents:
docs5 = ["doc12.txt", "doc18.txt", "doc20.txt"]
# Test 6
query6 = "Care sunt râurile mari din România?"

# Expected documents:
docs6 = ["doc30.txt", "doc19.txt", "doc28.txt"]
# Test 7
query7 = "Care sunt limbajele de programare populare?"
# Expected documents:
docs7 = ["doc12.txt", "doc20.txt", "doc14.txt"]

# Test 8
query8 = "Ce poți vizita în Transilvania?"
# Expected documents:
docs8 = ["doc21.txt", "doc23.txt", "doc13.txt"]

# Test 9
query9 = "Cine sunt compozitorii celebri din România?"
# Expected documents:
docs9 = ["doc22.txt", "doc25.txt", "doc27.txt"]

# Test 10
query10 = "Ce înseamnă învățarea automată?"
# Expected documents:
docs10 = ["doc18.txt", "doc12.txt", "doc20.txt"]

# Test 11
query11 = "Care sunt tradițiile românești importante?"
# Expected documents:
docs11 = ["doc27.txt", "doc17.txt", "doc11.txt"]

# Test 12
query12 = "Ce este Bucureștiul cunoscut ca?"
# Expected documents:
docs12 = ["doc16.txt", "doc26.txt", "doc17.txt"]

# Test 13
query13 = "Ce castele sunt faimoase în România?"
# Expected documents:
docs13 = ["doc23.txt", "doc21.txt", "doc19.txt"]

# Test 14
query14 = "Ce lacuri faimoase există în România?"
# Expected documents:
docs14 = ["doc28.txt", "doc29.txt", "doc30.txt"]

# Test 15
query15 = "Cine a scris Rapsodia Română?"
# Expected documents:
docs15 = ["doc22.txt", "doc25.txt", "doc27.txt"]

# Test 16
query16 = "Ce poți spune despre Delta Dunării?"
# Expected documents:
docs16 = ["doc19.txt", "doc30.txt", "doc28.txt"]

queries = [query1, query2, query3, query4, query5, query6, query7, query8, query9, query10, query11, query12, query13, query14, query15, query16]
docs = [docs1, docs2, docs3, docs4, docs5, docs6, docs7, docs8, docs9, docs10, docs11, docs12, docs13, docs14, docs15, docs16]

# Test performance normal IR system

In [32]:
#### SEARCH QUERY #####
top1_correct = 0
top3_correct = 0
total_queries = len(queries)

for query, doc in zip(queries, docs):
    results = ir_system.search(query)
    
    # Compute Top-1 Accuracy
    if results[0] in doc:  # Check if the first result is in the expected results
        top1_correct += 1
    
    # Compute Top-3 Accuracy
    top3_matches = [result for result in results[:3] if result in doc]  # Check top 3 results
    if top3_matches:
        top3_correct += 1
    
    # Print details for debugging
    print(f"Query: {query}")
    print(f"IR results: {results}")
    print(f"Expected results: {doc}")
    print(f"Top-3 Matches: {top3_matches}\n")

# Calculate accuracy
top1_accuracy = top1_correct / total_queries
top3_accuracy = top3_correct / total_queries

# Print overall performance
print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
print(f"Top-3 Accuracy: {top3_accuracy * 100:.2f}%")


Query: Ce pot vizita în România?
IR results: ['doc11.txt', 'doc19.txt', 'doc27.txt']
Expected results: ['doc11.txt', 'doc13.txt', 'doc19.txt']
Top-3 Matches: ['doc11.txt', 'doc19.txt']

Query: Care sunt mâncărurile tradiționale românești?
IR results: ['doc17.txt']
Expected results: ['doc17.txt', 'doc27.txt', 'doc11.txt']
Top-3 Matches: ['doc17.txt']

Query: Cine sunt artiștii români celebri?
IR results: ['doc22.txt', 'doc25.txt', 'doc13.txt']
Expected results: ['doc22.txt', 'doc25.txt', 'doc27.txt']
Top-3 Matches: ['doc22.txt', 'doc25.txt']

Query: Ce știi despre Castelul Bran și Dracula?
IR results: ['doc23.txt', 'doc17.txt', 'doc15.txt']
Expected results: ['doc23.txt', 'doc21.txt', 'doc19.txt']
Top-3 Matches: ['doc23.txt']

Query: Care este limbajul de programare utilizat în învățarea automată?
IR results: ['doc12.txt', 'doc20.txt', 'doc18.txt']
Expected results: ['doc12.txt', 'doc18.txt', 'doc20.txt']
Top-3 Matches: ['doc12.txt', 'doc20.txt', 'doc18.txt']

Query: Care sunt râurile m

# Test performance 1. Query Processing

In [48]:
# 1. Use the LLM the enhance the user's query
top1_correct = 0
top3_correct = 0
total_queries = len(queries)

for idx in range(len(queries)):
    query = queries[idx]
    doc = docs[idx]
    enhanced_query = query_enhance(query)
    results = ir_system.search(enhanced_query)
    
    # Compute Top-1 Accuracy
    top1_result = results[0] in doc
    if top1_result:  # Check if the first result is in the expected results
        top1_correct += 1
    
    # Compute Top-3 Accuracy
    top3_matches = [result for result in results[:3] if result in doc]  # Check top 3 results
    if top3_matches:
        top3_correct += 1
    
    # Print details for debugging
    print(f"Query: {query}")
    print(f"Enhanced query: {enhanced_query}")
    print(f"IR results: {results}")
    print(f"Expected results: {doc}")
    print(f"Top-1 Match: {top1_result}")
    print(f"Top-3 Matches: {top3_matches}")
    print(f"==============================================")

# Calculate accuracy
top1_accuracy = top1_correct / total_queries
top3_accuracy = top3_correct / total_queries

# Print overall performance
print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
print(f"Top-3 Accuracy: {top3_accuracy * 100:.2f}%")

    

Query: Ce pot vizita în România?
Enhanced query: Întrebare reformulată: Care sunt atracțiile turistice, destinațiile culturale și experiențele unice pe care le pot avea în timpul vizitei mele în România?
IR results: ['doc27.txt', 'doc23.txt', 'doc11.txt']
Expected results: ['doc11.txt', 'doc13.txt', 'doc19.txt']
Top-3 Matches: ['doc11.txt']
Query: Care sunt mâncărurile tradiționale românești?
Enhanced query: Care sunt felurile de mâncare tradiționale românești care sunt servite în mod obișnuit în diferite regiuni și care prezintă aromele și ingrediente unice ale bucătăriei românești?
IR results: ['doc17.txt', 'doc19.txt', 'doc27.txt']
Expected results: ['doc17.txt', 'doc27.txt', 'doc11.txt']
Top-3 Matches: ['doc17.txt', 'doc27.txt']
Query: Cine sunt artiștii români celebri?
Enhanced query: Cine sunt câțiva artiști români renumiți, inclusiv muzicieni, pictori, scriitori, actori și artiști vizuali, care au contribuit în mod semnificativ la lumea culturii și artei românești?
IR results: [